# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
from io import StringIO
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
#filepath = os.getcwd() + 'event_data'
filepath = os.path.abspath(os.curdir + '/../') + '/data/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/woox/Develop/datadiver-cassandra/src


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

targetfile = os.path.abspath(os.curdir + '/../') + '/data/target/event_data.csv'

with open(targetfile, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open(targetfile, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="../images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

# To establish connection and begin executing queries, need a session
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [92]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [93]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Create Pandas DataFrame from CSV

In [94]:
event_data = pd.read_csv(targetfile)
event_data.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,The Killers,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",594,Read My Mind,25
1,Tamia,Jayden,M,33,Graves,243.09506,paid,"Marinette, WI-MI",594,Officially Missing You (Radio Version),25
2,Randy Crawford,Jayden,M,34,Graves,270.75873,paid,"Marinette, WI-MI",594,Almaz,25
3,Frumpies,Jayden,M,35,Graves,134.47791,paid,"Marinette, WI-MI",594,Hello Kitty,25
4,Julia Fordham,Jayden,M,36,Graves,279.50975,paid,"Marinette, WI-MI",594,Girlfriend,25


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [95]:
query_by_session = """
SELECT artist, song, length
FROM songplay_by_session
WHERE sessionId = 338 AND itemInSession = 4
"""

query_by_user = """
SELECT artist, song, firstName, lastName
FROM songplay_by_user
WHERE userId = 10 AND sessionId = 182
ORDER BY itemInSession
"""

query_by_song = """
SELECT firstName, lastName
FROM songplay_by_song
WHERE song = 'All Hands Against His Own'
"""

In [96]:
query = """
CREATE TABLE IF NOT EXISTS songplay_by_session
(artist text, song text, length float, sessionId int,
itemInSession int, PRIMARY KEY (sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [97]:
query = """
CREATE TABLE IF NOT EXISTS songplay_by_user
(artist text, song text, firstName text, lastName text,
userId int, sessionId int, itemInSession int,
PRIMARY KEY (userId, sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [98]:
query = """
CREATE TABLE IF NOT EXISTS songplay_by_song
(firstName text, lastName text, song text,
PRIMARY KEY (song, firstName, lastName))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [99]:
class Batch:

    SESSION = 0
    USER = 1
    SONG = 2

    def __init__(self, session, max_size):
        self.session = session
        self.max_size = max_size
        self.size = 0

        self.batches = [None, None, None]

    def __begin_batch(self):
        for index, batch in enumerate(self.batches):
            self.batches[index] = StringIO()
            self.batches[index].write("BEGIN BATCH\n")

    def __create_insert(self, table, columns, values):
        query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({', '.join(values)});\n"
        return query

    def __end_batch(self):
        for index, batch in enumerate(self.batches):
            try:
                self.batches[index].write('APPLY BATCH;\n')
                self.batches[index].seek(0)
                self.session.execute(self.batches[index].read())
                self.batches[index].close()
            except Exception as e:
                print(e)
        
        self.batches = [None, None, None]
        self.size = 0
            

    def __read_songplay(self, songplay_line):
        songplay_line[0] = songplay_line[0].replace("'", "")
        songplay_line[9] = songplay_line[9].replace("'", "")

        return {
            'artist': f"'{songplay_line[0]}'",
            'firstName': f"'{songplay_line[1]}'",
            'gender': f"'{songplay_line[2]}'",
            'itemInSession': songplay_line[3],
            'lastName': f"'{songplay_line[4]}'",
            'length': songplay_line[5],
            'level': f"'{songplay_line[6]}'",
            'location': f"'{songplay_line[7]}'",
            'sessionId': songplay_line[8],
            'song': f"'{songplay_line[9]}'",
            'userId': songplay_line[10]
        }

    def __write(self, songplay):

        self.batches[Batch.SESSION].write(self.__create_insert(
            'songplay_by_session',
            ['artist', 'song', 'length', 'sessionId', 'itemInSession'],
            [songplay['artist'], songplay['song'], songplay['length'],
             songplay['sessionId'], songplay['itemInSession']])
        )

        self.batches[Batch.USER].write(self.__create_insert(
            'songplay_by_user',
            ['artist', 'song', 'firstName', 'lastName',
                'userId', 'sessionId', 'itemInSession'],
            [songplay['artist'], songplay['song'], songplay['firstName'], songplay['lastName'],
             songplay['userId'], songplay['sessionId'], songplay['itemInSession']])
        )

        self.batches[Batch.SONG].write(self.__create_insert(
            'songplay_by_song',
            ['firstName', 'lastName', 'song'],
            [songplay['firstName'], songplay['lastName'], songplay['song']])
        )
    
    def commit(self, close=False):
        self.__end_batch()
        if close:
            self.session.shutdown()
            self.session.close()
            self.session = None

    def insert(self, songplay_line):
        songplay = self.__read_songplay(songplay_line)

        if self.size == 0:
            self.__begin_batch()
        
        if self.size < self.max_size:
            self.__write(songplay)
            self.size += 1
        else:
            self.__write(songplay)
            self.__end_batch()

In [100]:
batch = Batch(session, 100)

with open(targetfile, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        batch.insert(line)
    
    batch.commit()

#### Do a SELECT to verify that the data have been inserted into each table

In [101]:
try:
    response = session.execute(query_by_session)
except Exception as e:
    print(e)

for row in response.all():
    print(row) 

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [102]:
try:
    response = session.execute(query_by_user)
except Exception as e:
    print(e)

for row in response.all():
    print(row)         

Row(artist='Down To The Bone', song='Keep On Keepin On', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [103]:
try:
    response = session.execute(query_by_song)
except Exception as e:
    print(e)

for row in response.all():
    print(row)                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [ ]:
query = "DROP TABLE IF EXISTS songplay_by_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songplay_by_user"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songplay_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()